In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('../../../Datos/Brutos/GRD_PUBLICO_2019.txt', sep = '|', dtype = str)
df2 = pd.read_csv('../../../Datos/Brutos/GRD_PUBLICO_2020.txt', sep = '|', dtype = str)
df3 = pd.DataFrame()

# Acá insertar la ruta de los dataset obtenidos de FONASA, no deben tener ningun procesamiento para que este script se ejecute correctamente

# https://www.fonasa.cl/sites/fonasa/datos-abiertos/bases-grd

In [ ]:
cols = [
    'SEXO', 'FECHA_NACIMIENTO', 'FECHAALTA',
    'DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9', 'DIAGNOSTICO10',
    'DIAGNOSTICO11','DIAGNOSTICO12','DIAGNOSTICO13','DIAGNOSTICO14','DIAGNOSTICO15','DIAGNOSTICO16','DIAGNOSTICO17','DIAGNOSTICO18','DIAGNOSTICO19','DIAGNOSTICO20',
    'DIAGNOSTICO21','DIAGNOSTICO22','DIAGNOSTICO23','DIAGNOSTICO24','DIAGNOSTICO25','DIAGNOSTICO26','DIAGNOSTICO27','DIAGNOSTICO28','DIAGNOSTICO29','DIAGNOSTICO30',
    "DIAGNOSTICO31","DIAGNOSTICO32","DIAGNOSTICO33","DIAGNOSTICO34","DIAGNOSTICO35",
    'PROCEDIMIENTO1','PROCEDIMIENTO2','PROCEDIMIENTO3','PROCEDIMIENTO4','PROCEDIMIENTO5','PROCEDIMIENTO6','PROCEDIMIENTO7','PROCEDIMIENTO8','PROCEDIMIENTO9','PROCEDIMIENTO10',
    'PROCEDIMIENTO11','PROCEDIMIENTO12','PROCEDIMIENTO13','PROCEDIMIENTO14','PROCEDIMIENTO15','PROCEDIMIENTO16','PROCEDIMIENTO17','PROCEDIMIENTO18','PROCEDIMIENTO19','PROCEDIMIENTO20',
    'PROCEDIMIENTO21','PROCEDIMIENTO22','PROCEDIMIENTO23','PROCEDIMIENTO24','PROCEDIMIENTO25','PROCEDIMIENTO26','PROCEDIMIENTO27','PROCEDIMIENTO28','PROCEDIMIENTO29','PROCEDIMIENTO30',
    'IR_29301_COD_GRD'
]

In [ ]:
df3 = pd.concat([df1[cols], df2[cols]])

In [ ]:
df3.replace(to_replace = ['DESCONOCIDO', '--'], value = None, inplace = True)
df3.dropna(subset = ['SEXO','FECHA_NACIMIENTO', 'FECHAALTA','DIAGNOSTICO1','PROCEDIMIENTO1','IR_29301_COD_GRD'], inplace = True)
df3.reset_index(inplace = True)

In [ ]:
import matplotlib.pyplot as plt

# asumiendo que 'df' es tu DataFrame
non_null_counts = df3.count()

In [ ]:
# plt.figure(figsize=(15, 15), dpi = 300)
# plt.bar(non_null_counts.index, non_null_counts.values)
# plt.xticks(rotation=90)  # rotar los nombres de las columnas para una mejor visualización
# plt.title('Cantidad de valores no nulos por columna')
# plt.xlabel('Columnas')
# plt.ylabel('Cantidad de valores no nulos')
# plt.show()

In [ ]:
df3.insert(loc = len(df3.columns), column = 'CDM', value = df3['IR_29301_COD_GRD'].str[0:2])
df3.insert(loc = len(df3.columns), column = 'GRDTIPO', value = df3['IR_29301_COD_GRD'].str[2:3])
df3.insert(loc = len(df3.columns), column = 'GRD', value = df3['IR_29301_COD_GRD'].str[3:5])
df3.insert(loc = len(df3.columns), column = 'SEVERIDAD', value = df3['IR_29301_COD_GRD'].str[5])

In [ ]:
from datetime import datetime

def age(vec):
    born = datetime.strptime(vec[0], "%Y-%m-%d").date()
    today = datetime.strptime(vec[1], "%Y-%m-%d").date()
    return today.year - born.year - ((today.month, 
                                      today.day) < (born.month, 
                                                    born.day))

In [ ]:
df3['FECHA_NACIMIENTO'] = df3[['FECHA_NACIMIENTO', 'FECHAALTA']].apply(age, axis = 1)

In [ ]:
df3 = df3.drop('FECHAALTA', axis = 1)

In [ ]:
cols = [
    'SEXO', 'FECHA_NACIMIENTO',
    'DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9', 'DIAGNOSTICO10',
    'DIAGNOSTICO11','DIAGNOSTICO12','DIAGNOSTICO13','DIAGNOSTICO14','DIAGNOSTICO15','DIAGNOSTICO16','DIAGNOSTICO17','DIAGNOSTICO18','DIAGNOSTICO19','DIAGNOSTICO20',
    'DIAGNOSTICO21','DIAGNOSTICO22','DIAGNOSTICO23','DIAGNOSTICO24','DIAGNOSTICO25','DIAGNOSTICO26','DIAGNOSTICO27','DIAGNOSTICO28','DIAGNOSTICO29','DIAGNOSTICO30',
    "DIAGNOSTICO31","DIAGNOSTICO32","DIAGNOSTICO33","DIAGNOSTICO34","DIAGNOSTICO35",
    'PROCEDIMIENTO1','PROCEDIMIENTO2','PROCEDIMIENTO3','PROCEDIMIENTO4','PROCEDIMIENTO5','PROCEDIMIENTO6','PROCEDIMIENTO7','PROCEDIMIENTO8','PROCEDIMIENTO9','PROCEDIMIENTO10',
    'PROCEDIMIENTO11','PROCEDIMIENTO12','PROCEDIMIENTO13','PROCEDIMIENTO14','PROCEDIMIENTO15','PROCEDIMIENTO16','PROCEDIMIENTO17','PROCEDIMIENTO18','PROCEDIMIENTO19','PROCEDIMIENTO20',
    'PROCEDIMIENTO21','PROCEDIMIENTO22','PROCEDIMIENTO23','PROCEDIMIENTO24','PROCEDIMIENTO25','PROCEDIMIENTO26','PROCEDIMIENTO27','PROCEDIMIENTO28','PROCEDIMIENTO29','PROCEDIMIENTO30',
]

In [ ]:
df3['FECHA_NACIMIENTO'].replace(to_replace = '-1', value = None, inplace = True)
df3['CDM'].replace(to_replace = '99', value = None, inplace = True)
df3['SEVERIDAD'].replace(to_replace = '9', value = None, inplace = True)
df3.dropna(subset = ['CDM','GRDTIPO', 'SEVERIDAD', 'FECHA_NACIMIENTO'], inplace = True)
df3.reset_index(inplace = True)

In [ ]:
grds = [
    'IR_29301_COD_GRD', 'CDM', 'GRDTIPO', 'GRD', 'SEVERIDAD'
]

In [ ]:
data = df3.astype("category")

In [ ]:
categorias_GRD = dict(zip( data['IR_29301_COD_GRD'].cat.codes, data['IR_29301_COD_GRD'] ))

In [ ]:
import json

salida = open("codes.json","w")
salida.write(json.dumps(categorias_GRD))
salida.close()
codedf = pd.read_json("codes.json",orient="index")
codedf.to_csv("codes.csv",header=["GRD"],index_label="codigo")

In [ ]:
x = data[cols]
y_total     = data[grds]
y_CDM       = data[['CDM']]
y_GRDTIPO   = data[['GRDTIPO']]
y_GRD       = data[['GRD']]
y_SEVERIDAD = data[['SEVERIDAD']]

In [ ]:
x = x.apply(lambda x: x.cat.codes)
y_total_cat     = y_total.apply(lambda x: x.cat.codes)
y_CDM_cat       = y_CDM.apply(lambda x: x.cat.codes)
y_GRDTIPO_cat   = y_GRDTIPO.apply(lambda x: x.cat.codes)
y_GRD_cat       = y_GRD.apply(lambda x: x.cat.codes)
y_SEVERIDAD_cat = y_SEVERIDAD.apply(lambda x: x.cat.codes)

In [ ]:
x_values = x.iloc[:, 0:69].to_numpy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

labelEncoder_x = LabelEncoder()

x_values[:, 0] = labelEncoder_x.fit_transform(x_values[:, 0])

ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(categories='auto'), [0])], remainder='passthrough'
)

x_values = ct.fit_transform(x_values)

In [ ]:
y_sev_dummies = pd.get_dummies(y_SEVERIDAD["SEVERIDAD"].astype(int), columns = y_SEVERIDAD["SEVERIDAD"].astype(int).unique(), dtype=int)
y_cdm_dummies = pd.get_dummies(y_CDM["CDM"].astype(int), columns = y_CDM["CDM"].astype(int).unique(), dtype=int)
y_grd_dummies = pd.get_dummies(y_GRD["GRD"].astype(int), columns = y_GRD["GRD"].astype(int).unique(), dtype=int)
y_grdtipo_dummies = pd.get_dummies(y_GRDTIPO["GRDTIPO"].astype(int), columns = y_GRDTIPO["GRDTIPO"].astype(int).unique(), dtype=int)

In [ ]:
new_cols = [
    'SEXO1', 'SEXO2', 'FECHA_NACIMIENTO',
    'DIAGNOSTICO1', 'DIAGNOSTICO2', 'DIAGNOSTICO3', 'DIAGNOSTICO4', 'DIAGNOSTICO5', 'DIAGNOSTICO6', 'DIAGNOSTICO7', 'DIAGNOSTICO8', 'DIAGNOSTICO9', 'DIAGNOSTICO10',
    'DIAGNOSTICO11','DIAGNOSTICO12','DIAGNOSTICO13','DIAGNOSTICO14','DIAGNOSTICO15','DIAGNOSTICO16','DIAGNOSTICO17','DIAGNOSTICO18','DIAGNOSTICO19','DIAGNOSTICO20',
    'DIAGNOSTICO21','DIAGNOSTICO22','DIAGNOSTICO23','DIAGNOSTICO24','DIAGNOSTICO25','DIAGNOSTICO26','DIAGNOSTICO27','DIAGNOSTICO28','DIAGNOSTICO29','DIAGNOSTICO30',
    "DIAGNOSTICO31","DIAGNOSTICO32","DIAGNOSTICO33","DIAGNOSTICO34","DIAGNOSTICO35",
    'PROCEDIMIENTO1','PROCEDIMIENTO2','PROCEDIMIENTO3','PROCEDIMIENTO4','PROCEDIMIENTO5','PROCEDIMIENTO6','PROCEDIMIENTO7','PROCEDIMIENTO8','PROCEDIMIENTO9','PROCEDIMIENTO10',
    'PROCEDIMIENTO11','PROCEDIMIENTO12','PROCEDIMIENTO13','PROCEDIMIENTO14','PROCEDIMIENTO15','PROCEDIMIENTO16','PROCEDIMIENTO17','PROCEDIMIENTO18','PROCEDIMIENTO19','PROCEDIMIENTO20',
    'PROCEDIMIENTO21','PROCEDIMIENTO22','PROCEDIMIENTO23','PROCEDIMIENTO24','PROCEDIMIENTO25','PROCEDIMIENTO26','PROCEDIMIENTO27','PROCEDIMIENTO28','PROCEDIMIENTO29','PROCEDIMIENTO30',
]

In [ ]:
x_values_df = pd.DataFrame(x_values.astype(int), columns = [new_cols], index = None, dtype = str)

In [ ]:
x_values_df.to_csv("../../../Datos/Procesados/x_values.csv", index = False)
y_cdm_dummies.to_csv("../../../Datos/Procesados/y_dummies_cdm.csv", index = False)
y_grdtipo_dummies.to_csv("../../../Datos/Procesados/y_dummies_grdtipo.csv", index = False)
y_grd_dummies.to_csv("../../../Datos/Procesados/y_dummies_grd.csv", index = False)
y_sev_dummies.to_csv("../../../Datos/Procesados/y_dummies_sev.csv", index = False)

# Data que necesita luego el script ANN.ipynb